```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import Drifter, CPUDrifter, GPUDrifter
from SWESimulators import DataAssimilationUtils as dautils

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

## Running multiple Simulators with different wind direction

While keeping a track on the drifters...

In this case, each particle is a combination of a unique drifter living in a unique model state.

Ideas that might evolve into code:
- It should be optional for a Drifter to have an observation. An additional `.hasObservation()` method needs to be implemented, and included somewhere.
- Does it make sense to have a set of Drifters without an Ocean model? They could live in a steady state environment where the u-v-fields are given from a netCDF? Or simply just from a numpy.array. 

In [ ]:
if 'ensemble' in globals():
    ensemble.cleanUp()

reload(Drifter)
reload(GPUDrifter)
reload(CDKLM16)
reload(Common)
reload(dautils)

class OceanStateWithDrifters:
    def __init__(self, sim, drifters):
        self.sim = sim
        assert isinstance(drifters, Drifter), 'drifters is type: ' + type(drifters) + \
                ', but has to be a Drifter-type instance.'
            
        self.drifters = drifters
    
    

class OceanStateEnsamble:
    def __init__(self, numParticles, cl_ctx):
        
        self.cl_ctx = cl_ctx
        
        self.numParticles = numParticles
        self.particles = [None]*(self.numParticles + 1)
        
        self.obs_index = self.numParticles
        
        self.simType = 'CDKLM16'
        
    def cleanUp(self):
        for oceanState in self.particles:
            if oceanState is not None:
                oceanState.cleanUp()
        
    def setGridInfo(self, nx, ny, dx, dy, dt, boundaryConditions):
        self.nx = nx
        self.ny = ny
        self.dx = dx
        self.dy = dy
        self.dt = dt
        
        self.observation_variance = 5*dx
        
        self.initialization_variance = 10*dx
        self.midPoint = 0.5*np.array([self.nx*self.dx, self.ny*self.dy])
        self.initialization_cov = np.eye(2)*self.initialization_variance
        
        self.boundaryConditions = boundaryConditions
        
        assert(self.simType == 'CDKLM16'), 'CDKLM16 is currently the only supported scheme'
        #if self.simType == 'CDKLM16':
        self.ghostCells = np.array([2,2,2,2])
        if self.boundaryConditions.isSponge():
            sponge = self.boundaryConditions.getSponge()
            for i in range(4):
                if sponge[i] > 0: 
                    self.ghostCells[i] = sponge[i]
        dataShape =  ( ny + self.ghostCells[0] + self.ghostCells[2], 
                       nx + self.ghostCells[1] + self.ghostCells[3]  )
            
        # Create base initial data:
        self.base_eta = np.zeros(dataShape, dtype=np.float32, order='C')
        self.base_hu  = np.zeros(dataShape, dtype=np.float32, order='C');
        self.base_hv  = np.zeros(dataShape, dtype=np.float32, order='C');
        
        # Bathymetry:
        waterDepth = 5
        self.base_Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterDepth
 
    
    def setParameters(self, f, g=9.81, beta=0, r=0, wind=Common.WindStressParams(type=99)):
        self.g = g
        self.f = f
        self.beta = beta
        self.r = r
        self.wind = wind
    
    def initWindCase(self, driftersPerOceanModel=1):
        self.windSpeed = 2.0
        self.directions = np.random.rand(self.numParticles + 1)*360
        print "Directions: ", self.directions
        self.driftersPerOceanModel = driftersPerOceanModel
        
        for i in range(self.numParticles+1):
            wind = Common.WindStressParams(type=50, 
                                           wind_speed=self.windSpeed,
                                           wind_direction=self.directions[i])

            
            self.particles[i] = CDKLM16.CDKLM16(self.cl_ctx, \
                                                self.base_eta, self.base_hu, self.base_hv, \
                                                self.base_Hi, \
                                                self.nx, self.ny, self.dx, self.dy, self.dt, \
                                                self.g, self.f, self.r, \
                                                wind_stress=wind, \
                                                boundary_conditions=self.boundaryConditions, \
                                                write_netcdf=False)
            if i == self.numParticles:
                # All particles done, only the observation is left,
                # and for the observation we only use one drifter, regardless of the
                # number in the other particles.
                driftersPerOceanModel = 1
            
            drifters = GPUDrifter.GPUDrifter(cl_ctx, driftersPerOceanModel,
                                             observation_variance=self.observation_variance,
                                             boundaryConditions=self.boundaryConditions,
                                             domain_size_x=self.nx*self.dx, domain_size_y=self.ny*self.dy)
            initPos = np.random.multivariate_normal(self.midPoint, self.initialization_cov, driftersPerOceanModel)
            drifters.setParticlePositions(initPos)
            #print "drifter particles: ", drifter.getParticlePositions()
            #print "drifter observations: ", drifter.getObservationPosition()
            self.particles[i].attachDrifters(drifters)
        
    
    def getParticlePositions(self):
        drifterPositions = np.empty((0,2))
        for oceanState in self.particles[:-1]:
            drifterPositions = np.append(drifterPositions, 
                                         oceanState.drifters.getParticlePositions(),
                                         axis=0)
        return drifterPositions
    
    def getObservationPosition(self):
        observation = self.particles[self.obs_index].drifters.getParticlePositions()
        return observation[0,:]
    
    def step(self, t):
        simNo = 0
        for oceanState in self.particles:
            #print "Starting sim " + str(simNo)
            oceanState.step(t)
            #print "Finished sim " + str(simNo)      
            simNo = simNo + 1
    
    def getDistances(self, obs=None):
        if obs is None:
            obs = self.getObservationPosition()
        distances = np.empty(0)
        counter = 0
        for oceanState in self.particles[:-1]:
            distancesFromOceanState = oceanState.drifters.getDistances(obs)
            distances = np.append(distances,
                                  distancesFromOceanState,
                                  axis=0)
            counter += 1
        return distances
            
    def printMaxOceanStates(self):
        simNo = 0
        for oceanState in self.particles:
            eta, hu, hv = oceanState.download()
            print "------- simNo: " + str(simNo) + " -------"
            print "t = " + str(oceanState.t)
            print "Max eta: ", np.max(eta)
            print "Max hu:  ", np.max(hu)
            print "Max hv:  ", np.max(hv)
            simNo = simNo + 1
            
    def resample(self, newSampleIndices, reinitialization_variance):
        positions = self.getParticlePositions()
        windDirection = self.directions
        newWindDirection = np.empty_like(windDirection)
        newPos = np.empty((self.driftersPerOceanModel, 2))
        newOceanStates = [None]*self.getNumParticles()
        for i in range(self.getNumParticles()):
            index = newSampleIndices[i]
            #print "(particle no, position, old direction, new direction): "
            newWindDirection[i] = np.random.normal(windDirection[index], reinitialization_variance, 1)
            if newWindDirection[i] > 360:
                newWindDirection[i] -= 360
            elif newWindDirection[i] < 0:
                newWindDirection[i] += 360
            newPos[:,:] = positions[index,:]
            #print "\t", (index, positions[index,:], windDirection[index])
            #print "\t", (index, newPos, newWindDirection[i])
            
            #newWindInstance = Common.WindStressParams()
            newWindInstance = Common.WindStressParams(type=50, 
                                                      wind_speed=self.windSpeed,
                                                      wind_direction=newWindDirection[i])
            
            # Download index's ocean state:
            eta, hu, hv = self.particles[index].download()
            newOceanStates[i] = (eta, hu, hv)
            
            self.particles[i].wind_stress = newWindInstance
            self.particles[i].drifters.setParticlePositions(newPos)

        self.directions = newWindDirection.copy()
        
        # New loop for transferring the correct ocean states back up to the GPU:
        for i in range(self.getNumParticles()):
            self.particles[i].upload(newOceanStates[i][0],
                                     newOceanStates[i][1],
                                     newOceanStates[i][2])
                    
            
    ### Duplication of code
    def getEnsembleMean(self):
        return None
    def getDomainSizeX(self):
        return self.nx*self.dx
    def getDomainSizeY(self):
        return self.ny*self.dy
    def getObservationVariance(self):
        return self.observation_variance
    def getNumParticles(self):
        return self.numParticles
    
    def plotDistanceInfo(self, title=None):
        """
        Utility function for generating informative plots of the ensemble relative to the observation
        """    
        fig = plt.figure(figsize=(10,6))
        gridspec.GridSpec(2, 3)
        
        # PLOT POSITIONS OF PARTICLES AND OBSERVATIONS
        ax0 = plt.subplot2grid((2,3), (0,0))
        plt.plot(self.getParticlePositions()[:,0], \
                 self.getParticlePositions()[:,1], 'b.')
        plt.plot(self.getObservationPosition()[0], \
                 self.getObservationPosition()[1], 'r.')
        ensembleMean = self.getEnsembleMean()
        if ensembleMean is not None:
            plt.plot(ensembleMean[0], ensembleMean[1], 'r+')
        plt.xlim(0, self.getDomainSizeX())
        plt.xlabel('x')
        plt.ylabel('y')
        plt.ylim(0, self.getDomainSizeY())
        plt.title("Particle positions")
        
        # PLOT DISCTRIBUTION OF PARTICLE DISTANCES AND THEORETIC OBSERVATION PDF
        ax0 = plt.subplot2grid((2,3), (0,1), colspan=2)
        distances = self.getDistances()
        obs_var = self.getObservationVariance()
        plt.hist(distances, bins=30, \
                 range=(0, max(min(self.getDomainSizeX(), self.getDomainSizeY()), np.max(distances))),\
                 normed=True, label="particle distances")
        
        # With observation 
        x = np.linspace(0, max(self.getDomainSizeX(), self.getDomainSizeY()), num=100)
        cauchy_pdf = dautils.getCauchyWeight(x, obs_var, normalize=False)
        gauss_pdf = dautils.getGaussianWeight(x, obs_var, normalize=False)
        plt.plot(x, cauchy_pdf, 'r', label="obs Cauchy pdf")
        plt.plot(x, gauss_pdf, 'g', label="obs Gauss pdf")
        plt.legend()
        plt.title("Distribution of particle distances from observation")
        
        # PLOT SORTED DISTANCES FROM OBSERVATION
        ax0 = plt.subplot2grid((2,3), (1,0), colspan=3)
        cauchyWeights = dautils.getCauchyWeight(distances, obs_var)
        gaussWeights = dautils.getGaussianWeight(distances, obs_var)
        indices_sorted_by_observation = distances.argsort()
        ax0.plot(cauchyWeights[indices_sorted_by_observation]/np.max(cauchyWeights), 'r', label="Cauchy weight")
        ax0.plot(gaussWeights[indices_sorted_by_observation]/np.max(gaussWeights), 'g', label="Gauss weight")
        ax0.set_ylabel('Relative weight')
        ax0.grid()
        ax0.set_ylim(0,1.4)
        plt.legend(loc=7)
        
        ax1 = ax0.twinx()
        ax1.plot(distances[indices_sorted_by_observation], label="distance")
        ax1.set_ylabel('Distance from observation', color='b')
        
        plt.title("Sorted distances from observation")

        if title is not None:
            plt.suptitle(title, fontsize=16)
            

In [ ]:
# Parameters
nx, ny, dx, dy = 50, 50, 4.0, 4.0
dt = 1
f = 0.0
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

In [ ]:
#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


In [ ]:
try:
    if 'ensemble' in globals():
        ensemble.cleanUp()
except TypeError:
    pass
        
numParticles = 30
ensemble = OceanStateEnsamble(numParticles, cl_ctx)
ensemble.setGridInfo(nx, ny, dx, dy, dt, boundaryConditions)
ensemble.setParameters(f)

driftersPerSim = 1
ensemble.initWindCase(driftersPerSim)
print "ensemble of " + str(numParticles) + " ocean models, each with " + str(driftersPerSim) + " drifter(s)."
startPos = ensemble.getParticlePositions()
startObs = ensemble.getObservationPosition()
#print "StartPos:\n", startPos
fig = plt.figure(figsize=(5,5))
plt.plot(startPos[:,0], startPos[:,1], 'mo')
plt.plot(startObs[0], startObs[1], 'co')
plt.ylim([0, ny*dy])
plt.xlim([0, nx*dx])

# initial model integration
#for i in range(10):
for i in range(5):
    print "iteration " + str(i)
    ensemble.step(100)
    pos = ensemble.getParticlePositions()
    plt.plot(pos[:,0], pos[:,1], 'r.')
    obs = ensemble.getObservationPosition()
    plt.plot(obs[0], obs[1], 'b.')
    #print pos

# Loop doing resampling + model integration
for pf in range(3):
    pos = ensemble.getParticlePositions()
    plt.plot(pos[:,0], pos[:,1], 'kx')
    obs = ensemble.getObservationPosition()
    plt.plot(obs[0], obs[1], 'kx')
    
    print "Resampling"
    dautils.probabilisticResampling(ensemble, reinitialization_variance=20)
    print "Resampling done"

    pos = ensemble.getParticlePositions()
    plt.plot(pos[:,0], pos[:,1], 'rs')
    obs = ensemble.getObservationPosition()
    plt.plot(obs[0], obs[1], 'bs')
    #for i in range(10):
    for i in range(5):
        print "iteration " + str(i)
        ensemble.step(20)
        pos = ensemble.getParticlePositions()
        plt.plot(pos[:,0], pos[:,1], 'r.')
        obs = ensemble.getObservationPosition()
        plt.plot(obs[0], obs[1], 'b.')
        #print pos

pos = ensemble.getParticlePositions()
plt.plot(pos[:,0], pos[:,1], 'rx')
obs = ensemble.getObservationPosition()
plt.plot(obs[0], obs[1], 'bx')

#distances = ensemble.getDistances()
#print distances
#print "len(distances): ", len(distances)
#print "ensemble.getParticlePositions().shape: ", ensemble.getParticlePositions().shape

ensemble.plotDistanceInfo()

In [ ]:

if 'ensemble' in globals():
    ensemble.cleanUp()

numParticles = 10
ensemble = OceanStateEnsamble(numParticles, cl_ctx)
ensemble.setGridInfo(nx, ny, dx, dy, dt, boundaryConditions)
ensemble.setParameters(f)

driftersPerSim = 1
ensemble.initWindCase(driftersPerSim)
print "ensemble of " + str(numParticles) + " ocean models, each with " + str(driftersPerSim) + " drifter(s)."

fig = plt.figure()
eta, hu, hv = ensemble.particles[0].download()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta[2:-2, 2:-2], hu[2:-2, 2:-2], hv[2:-2, 2:-2])
plotter.showParticles(ensemble.particles[0].drifters)

T = 30
def animate(i):
    if i <= T/2:
        print "iteration " + str(i)
        ensemble.step(30)

        eta, hu, hv = ensemble.particles[0].download()
        plotter.plot(eta[2:-2, 2:-2], hu[2:-2, 2:-2], hv[2:-2, 2:-2])
        plotter.showParticles(ensemble.particles[0].drifters)
        fig.suptitle("CDKLM16 before Particle filter on wind direction, it = " + str(i), fontsize=18)
        #print pos
    if i == T/2:
        print "Resampling"
        dautils.probabilisticResampling(ensemble, reinitialization_variance=0)
        print "Resampling done"

        eta, hu, hv = ensemble.particles[0].download()
        plotter.plot(eta[2:-2, 2:-2], hu[2:-2, 2:-2], hv[2:-2, 2:-2])
        plotter.showParticles(ensemble.particles[0].drifters)
        fig.suptitle("CDKLM16 at Particle filter on wind direction, it = " + str(i), fontsize=18)
        
    if i > T/2:
        print "iteration " + str(i)
        ensemble.step(30)
        eta, hu, hv = ensemble.particles[0].download()
        plotter.plot(eta[2:-2, 2:-2], hu[2:-2, 2:-2], hv[2:-2, 2:-2])
        plotter.showParticles(ensemble.particles[0].drifters)
        fig.suptitle("CDKLM16 after Particle filter on wind direction, it = " + str(i), fontsize=18)
        #print pos

anim = animation.FuncAnimation(fig, animate, range(T), interval=300)
plt.close(anim._fig)
anim

In [ ]:
print np.empty(0), len(np.empty(0))

print len(ensemble.particles[:-1])
ensemble.printMaxOceanStates()